****Ingest Website to Graph DB****

**Part 3** - Extract Entities & Relationships using Langchain

Extract Entities and Relatonships from a body of Text using langchain.

This is a GCP reworking of

https://python.langchain.com/docs/use_cases/graph/constructing/#llm-graph-transformer

This notebook seeks to become independant of the following packages.

*langchain-experimental*

*langchain-community*

Instead copying the relevant classes into notebook cells.  

In [ ]:
pip install -U langchain langchain-google-vertexai neo4j

**Check Version Nos of what was installed**

In [88]:
!pip show langchain langchain-core langchain-google-vertexai langchain-experimental langchain-community neo4j google-cloud-aiplatform

9037.69s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Name: langchain
Version: 0.1.20
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /opt/conda/lib/python3.10/site-packages
Requires: aiohttp, async-timeout, dataclasses-json, langchain-community, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 
---
Name: langchain-core
Version: 0.1.52
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /opt/conda/lib/python3.10/site-packages
Requires: jsonpatch, langsmith, packaging, pydantic, PyYAML, tenacity
Required-by: langchain, langchain-community, langchain-google-vertexai, langchain-text-splitters
---
Name: langchain-google-vertexai
Version: 1.0.3
Summary: An integration package connecting Google VertexAI and LangChain
Home-page: https://github.com/lang

**Check Jupyter Version No**

In [ ]:
!jupyter --version

**Check Python Version/Path** - *Expect 3.10.14*

In [ ]:
import sys
import platform
print(sys.version)
print(platform.python_version())
print(sys.path)

***OpenAI Detox***

Copy Required Code from langchain-experimental

Have retained langchain-community.

This permits detoxification (re refs to ChatGPT in command promt).

It also eliminates doubt as to what code is being executed wrt. versions etc. 

It also permits baci debuffing using print sttaements where needed. 

This is a copy of the entire libs\experimental\langchain_experimental\graph_transformers\llm.py file.

This has resulted in a far more stable outcome wrt invocation of LLMs for this purpose. 

In [ ]:
import asyncio
import json
from typing import Any, Dict, List, Optional, Sequence, Tuple, Type, Union, cast

from langchain_community.graphs.graph_document import GraphDocument, Node, Relationship
from langchain_core.documents import Document
from langchain_core.language_models import BaseLanguageModel
from langchain_core.messages import SystemMessage
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    PromptTemplate,
)
from langchain_core.pydantic_v1 import BaseModel, Field, create_model

examples = [
    {
        "text": (
            "Adam is a software engineer in Microsoft since 2009, "
            "and last year he got an award as the Best Talent"
        ),
        "head": "Adam",
        "head_type": "Person",
        "relation": "WORKS_FOR",
        "tail": "Microsoft",
        "tail_type": "Company",
    },
    {
        "text": (
            "Adam is a software engineer in Microsoft since 2009, "
            "and last year he got an award as the Best Talent"
        ),
        "head": "Adam",
        "head_type": "Person",
        "relation": "HAS_AWARD",
        "tail": "Best Talent",
        "tail_type": "Award",
    },
    {
        "text": (
            "Microsoft is a tech company that provide "
            "several products such as Microsoft Word"
        ),
        "head": "Microsoft Word",
        "head_type": "Product",
        "relation": "PRODUCED_BY",
        "tail": "Microsoft",
        "tail_type": "Company",
    },
    {
        "text": "Microsoft Word is a lightweight app that accessible offline",
        "head": "Microsoft Word",
        "head_type": "Product",
        "relation": "HAS_CHARACTERISTIC",
        "tail": "lightweight app",
        "tail_type": "Characteristic",
    },
    {
        "text": "Microsoft Word is a lightweight app that accessible offline",
        "head": "Microsoft Word",
        "head_type": "Product",
        "relation": "HAS_CHARACTERISTIC",
        "tail": "accessible offline",
        "tail_type": "Characteristic",
    },
]

system_prompt = (
    "# Knowledge Graph Instructions for Gemini\n"
    "## 1. Overview\n"
    "You are a top-tier algorithm designed for extracting information in structured "
    "formats to build a knowledge graph.\n"
    "Try to capture as much information from the text as possible without "
    "sacrifing accuracy. Do not add any information that is not explicitly "
    "mentioned in the text\n"
    "- **Nodes** represent entities and concepts.\n"
    "- The aim is to achieve simplicity and clarity in the knowledge graph, making it\n"
    "accessible for a vast audience.\n"
    "## 2. Labeling Nodes\n"
    "- **Consistency**: Ensure you use available types for node labels.\n"
    "Ensure you use basic or elementary types for node labels.\n"
    "- For example, when you identify an entity representing a person, "
    "always label it as **'person'**. Avoid using more specific terms "
    "like 'mathematician' or 'scientist'"
    "  - **Node IDs**: Never utilize integers as node IDs. Node IDs should be "
    "names or human-readable identifiers found in the text.\n"
    "- **Relationships** represent connections between entities or concepts.\n"
    "Ensure consistency and generality in relationship types when constructing "
    "knowledge graphs. Instead of using specific and momentary types "
    "such as 'BECAME_PROFESSOR', use more general and timeless relationship types "
    "like 'PROFESSOR'. Make sure to use general and timeless relationship types!\n"
    "## 3. Coreference Resolution\n"
    "- **Maintain Entity Consistency**: When extracting entities, it's vital to "
    "ensure consistency.\n"
    'If an entity, such as "John Doe", is mentioned multiple times in the text '
    'but is referred to by different names or pronouns (e.g., "Joe", "he"),'
    "always use the most complete identifier for that entity throughout the "
    'knowledge graph. In this example, use "John Doe" as the entity ID.\n'
    "Remember, the knowledge graph should be coherent and easily understandable, "
    "so maintaining consistency in entity references is crucial.\n"
    "## 4. Strict Compliance\n"
    "Adhere to the rules strictly. Non-compliance will result in termination."
)

default_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system_prompt,
        ),
        (
            "human",
            (
                "Tip: Make sure to answer in the correct format and do "
                "not include any explanations. "
                "Use the given format to extract information from the "
                "following input: {input}"
            ),
        ),
    ]
)


def _get_additional_info(input_type: str) -> str:
    # Check if the input_type is one of the allowed values
    if input_type not in ["node", "relationship", "property"]:
        raise ValueError("input_type must be 'node', 'relationship', or 'property'")

    # Perform actions based on the input_type
    if input_type == "node":
        return (
            "Ensure you use basic or elementary types for node labels.\n"
            "For example, when you identify an entity representing a person, "
            "always label it as **'Person'**. Avoid using more specific terms "
            "like 'Mathematician' or 'Scientist'"
        )
    elif input_type == "relationship":
        return (
            "Instead of using specific and momentary types such as "
            "'BECAME_PROFESSOR', use more general and timeless relationship types like "
            "'PROFESSOR'. However, do not sacrifice any accuracy for generality"
        )
    elif input_type == "property":
        return ""
    return ""


def optional_enum_field(
    enum_values: Optional[List[str]] = None,
    description: str = "",
    input_type: str = "node",
    **field_kwargs: Any,
) -> Any:
    """Utility function to conditionally create a field with an enum constraint."""
    if enum_values:
        return Field(
            ...,
            enum=enum_values,
            description=f"{description}. Available options are {enum_values}",
            **field_kwargs,
        )
    else:
        additional_info = _get_additional_info(input_type)
        return Field(..., description=description + additional_info, **field_kwargs)


class _Graph(BaseModel):
    nodes: Optional[List]
    relationships: Optional[List]


class UnstructuredRelation(BaseModel):
    head: str = Field(
        description=(
            "extracted head entity like Microsoft, Apple, John. "
            "Must use human-readable unique identifier."
        )
    )
    head_type: str = Field(
        description="type of the extracted head entity like Person, Company, etc"
    )
    relation: str = Field(description="relation between the head and the tail entities")
    tail: str = Field(
        description=(
            "extracted tail entity like Microsoft, Apple, John. "
            "Must use human-readable unique identifier."
        )
    )
    tail_type: str = Field(
        description="type of the extracted tail entity like Person, Company, etc"
    )


def create_unstructured_prompt(
    node_labels: Optional[List[str]] = None, rel_types: Optional[List[str]] = None
) -> ChatPromptTemplate:
    node_labels_str = str(node_labels) if node_labels else ""
    rel_types_str = str(rel_types) if rel_types else ""
    base_string_parts = [
        "You are a top-tier algorithm designed for extracting information in "
        "structured formats to build a knowledge graph. Your task is to identify "
        "the entities and relations requested with the user prompt from a given "
        "text. You must generate the output in a JSON format containing a list "
        'with JSON objects. Each object should have the keys: "head", '
        '"head_type", "relation", "tail", and "tail_type". The "head" '
        "key must contain the text of the extracted entity with one of the types "
        "from the provided list in the user prompt.",
        f'The "head_type" key must contain the type of the extracted head entity, '
        f"which must be one of the types from {node_labels_str}."
        if node_labels
        else "",
        f'The "relation" key must contain the type of relation between the "head" '
        f'and the "tail", which must be one of the relations from {rel_types_str}.'
        if rel_types
        else "",
        f'The "tail" key must represent the text of an extracted entity which is '
        f'the tail of the relation, and the "tail_type" key must contain the type '
        f"of the tail entity from {node_labels_str}."
        if node_labels
        else "",
        "Attempt to extract as many entities and relations as you can. Maintain "
        "Entity Consistency: When extracting entities, it's vital to ensure "
        'consistency. If an entity, such as "John Doe", is mentioned multiple '
        "times in the text but is referred to by different names or pronouns "
        '(e.g., "Joe", "he"), always use the most complete identifier for '
        "that entity. The knowledge graph should be coherent and easily "
        "understandable, so maintaining consistency in entity references is "
        "crucial.",
        "IMPORTANT NOTES:\n- Don't add any explanation and text.",
    ]
    system_prompt = "\n".join(filter(None, base_string_parts))

    system_message = SystemMessage(content=system_prompt)
    parser = JsonOutputParser(pydantic_object=UnstructuredRelation)

    human_prompt = PromptTemplate(
        template="""Based on the following example, extract entities and 
relations from the provided text.\n\n
Use the following entity types, don't use other entity that is not defined below:
# ENTITY TYPES:
{node_labels}

Use the following relation types, don't use other relation that is not defined below:
# RELATION TYPES:
{rel_types}

Below are a number of examples of text and their extracted entities and relationships.
{examples}

For the following text, extract entities and relations as in the provided example.
{format_instructions}\nText: {input}""",
        input_variables=["input"],
        partial_variables={
            "format_instructions": parser.get_format_instructions(),
            "node_labels": node_labels,
            "rel_types": rel_types,
            "examples": examples,
        },
    )

    human_message_prompt = HumanMessagePromptTemplate(prompt=human_prompt)

    chat_prompt = ChatPromptTemplate.from_messages(
        [system_message, human_message_prompt]
    )
    return chat_prompt


def create_simple_model(
    node_labels: Optional[List[str]] = None,
    rel_types: Optional[List[str]] = None,
    node_properties: Union[bool, List[str]] = False,
) -> Type[_Graph]:
    """
    Simple model allows to limit node and/or relationship types.
    Doesn't have any node or relationship properties.
    """

    node_fields: Dict[str, Tuple[Any, Any]] = {
        "id": (
            str,
            Field(..., description="Name or human-readable unique identifier."),
        ),
        "type": (
            str,
            optional_enum_field(
                node_labels,
                description="The type or label of the node.",
                input_type="node",
            ),
        ),
    }
    if node_properties:
        if isinstance(node_properties, list) and "id" in node_properties:
            raise ValueError("The node property 'id' is reserved and cannot be used.")
        # Map True to empty array
        node_properties_mapped: List[str] = (
            [] if node_properties is True else node_properties
        )

        class Property(BaseModel):
            """A single property consisting of key and value"""

            key: str = optional_enum_field(
                node_properties_mapped,
                description="Property key.",
                input_type="property",
            )
            value: str = Field(..., description="value")

        node_fields["properties"] = (
            Optional[List[Property]],
            Field(None, description="List of node properties"),
        )
    SimpleNode = create_model("SimpleNode", **node_fields)  # type: ignore

    class SimpleRelationship(BaseModel):
        """Represents a directed relationship between two nodes in a graph."""

        source_node_id: str = Field(
            description="Name or human-readable unique identifier of source node"
        )
        source_node_type: str = optional_enum_field(
            node_labels,
            description="The type or label of the source node.",
            input_type="node",
        )
        target_node_id: str = Field(
            description="Name or human-readable unique identifier of target node"
        )
        target_node_type: str = optional_enum_field(
            node_labels,
            description="The type or label of the target node.",
            input_type="node",
        )
        type: str = optional_enum_field(
            rel_types,
            description="The type of the relationship.",
            input_type="relationship",
        )

    class DynamicGraph(_Graph):
        """Represents a graph document consisting of nodes and relationships."""

        nodes: Optional[List[SimpleNode]] = Field(description="List of nodes")  # type: ignore
        relationships: Optional[List[SimpleRelationship]] = Field(
            description="List of relationships"
        )

    return DynamicGraph


def map_to_base_node(node: Any) -> Node:
    """Map the SimpleNode to the base Node."""
    properties = {}
    if hasattr(node, "properties") and node.properties:
        for p in node.properties:
            properties[format_property_key(p.key)] = p.value
    return Node(id=node.id, type=node.type, properties=properties)


def map_to_base_relationship(rel: Any) -> Relationship:
    """Map the SimpleRelationship to the base Relationship."""
    source = Node(id=rel.source_node_id, type=rel.source_node_type)
    target = Node(id=rel.target_node_id, type=rel.target_node_type)
    return Relationship(source=source, target=target, type=rel.type)


def _parse_and_clean_json(
    argument_json: Dict[str, Any],
) -> Tuple[List[Node], List[Relationship]]:
    nodes = []
    for node in argument_json["nodes"]:
        if not node.get("id"):  # Id is mandatory, skip this node
            continue
        nodes.append(
            Node(
                id=node["id"],
                type=node.get("type"),
            )
        )
    relationships = []
    for rel in argument_json["relationships"]:
        # Mandatory props
        if (
            not rel.get("source_node_id")
            or not rel.get("target_node_id")
            or not rel.get("type")
        ):
            continue

        # Node type copying if needed from node list
        if not rel.get("source_node_type"):
            try:
                rel["source_node_type"] = [
                    el.get("type")
                    for el in argument_json["nodes"]
                    if el["id"] == rel["source_node_id"]
                ][0]
            except IndexError:
                rel["source_node_type"] = None
        if not rel.get("target_node_type"):
            try:
                rel["target_node_type"] = [
                    el.get("type")
                    for el in argument_json["nodes"]
                    if el["id"] == rel["target_node_id"]
                ][0]
            except IndexError:
                rel["target_node_type"] = None

        source_node = Node(
            id=rel["source_node_id"],
            type=rel["source_node_type"],
        )
        target_node = Node(
            id=rel["target_node_id"],
            type=rel["target_node_type"],
        )
        relationships.append(
            Relationship(
                source=source_node,
                target=target_node,
                type=rel["type"],
            )
        )
    return nodes, relationships


def _format_nodes(nodes: List[Node]) -> List[Node]:
    return [
        Node(
            id=el.id.title() if isinstance(el.id, str) else el.id,
            type=el.type.capitalize(),
            properties=el.properties,
        )
        for el in nodes
    ]


def _format_relationships(rels: List[Relationship]) -> List[Relationship]:
    return [
        Relationship(
            source=_format_nodes([el.source])[0],
            target=_format_nodes([el.target])[0],
            type=el.type.replace(" ", "_").upper(),
        )
        for el in rels
    ]


def format_property_key(s: str) -> str:
    words = s.split()
    if not words:
        return s
    first_word = words[0].lower()
    capitalized_words = [word.capitalize() for word in words[1:]]
    return "".join([first_word] + capitalized_words)


def _convert_to_graph_document(
    raw_schema: Dict[Any, Any],
) -> Tuple[List[Node], List[Relationship]]:
    # If there are validation errors
    if not raw_schema["parsed"]:
        try:
            try:  # OpenAI type response
                argument_json = json.loads(
                    raw_schema["raw"].additional_kwargs["tool_calls"][0]["function"][
                        "arguments"
                    ]
                )
            except Exception:  # Google type response
                argument_json = json.loads(
                    raw_schema["raw"].additional_kwargs["function_call"]["arguments"]
                )

            nodes, relationships = _parse_and_clean_json(argument_json)
        except Exception:  # If we can't parse JSON
            return ([], [])
    else:  # If there are no validation errors use parsed pydantic object
        parsed_schema: _Graph = raw_schema["parsed"]
        nodes = (
            [map_to_base_node(node) for node in parsed_schema.nodes]
            if parsed_schema.nodes
            else []
        )

        relationships = (
            [map_to_base_relationship(rel) for rel in parsed_schema.relationships]
            if parsed_schema.relationships
            else []
        )
    # Title / Capitalize
    return _format_nodes(nodes), _format_relationships(relationships)


class LLMGraphTransformer:
    """Transform documents into graph-based documents using a LLM.

    It allows specifying constraints on the types of nodes and relationships to include
    in the output graph. The class doesn't support neither extract and node or
    relationship properties

    Args:
        llm (BaseLanguageModel): An instance of a language model supporting structured
          output.
        allowed_nodes (List[str], optional): Specifies which node types are
          allowed in the graph. Defaults to an empty list, allowing all node types.
        allowed_relationships (List[str], optional): Specifies which relationship types
          are allowed in the graph. Defaults to an empty list, allowing all relationship
          types.
        prompt (Optional[ChatPromptTemplate], optional): The prompt to pass to
          the LLM with additional instructions.
        strict_mode (bool, optional): Determines whether the transformer should apply
          filtering to strictly adhere to `allowed_nodes` and `allowed_relationships`.
          Defaults to True.

    Example:
        .. code-block:: python
            from langchain_experimental.graph_transformers import LLMGraphTransformer
            from langchain_core.documents import Document
            from langchain_openai import ChatOpenAI

            llm=ChatOpenAI(temperature=0)
            transformer = LLMGraphTransformer(
                llm=llm,
                allowed_nodes=["Person", "Organization"])

            doc = Document(page_content="Elon Musk is suing OpenAI")
            graph_documents = transformer.convert_to_graph_documents([doc])
    """

    def __init__(
        self,
        llm: BaseLanguageModel,
        allowed_nodes: List[str] = [],
        allowed_relationships: List[str] = [],
        prompt: Optional[ChatPromptTemplate] = None,
        strict_mode: bool = True,
        node_properties: Union[bool, List[str]] = False,
    ) -> None:
        self.allowed_nodes = allowed_nodes
        self.allowed_relationships = allowed_relationships
        self.strict_mode = strict_mode
        self._function_call = True
        # Check if the LLM really supports structured output
        try:
            llm.with_structured_output(_Graph)
        except NotImplementedError:
            self._function_call = False
        if not self._function_call:
            if node_properties:
                raise ValueError(
                    "The 'node_properties' parameter cannot be used "
                    "in combination with a LLM that doesn't support "
                    "native function calling."
                )
            try:
                import json_repair

                self.json_repair = json_repair
            except ImportError:
                raise ImportError(
                    "Could not import json_repair python package. "
                    "Please install it with `pip install json-repair`."
                )
            prompt = prompt or create_unstructured_prompt(
                allowed_nodes, allowed_relationships
            )
            self.chain = prompt | llm
        else:
            # Define chain
            schema = create_simple_model(
                allowed_nodes, allowed_relationships, node_properties
            )
            structured_llm = llm.with_structured_output(schema, include_raw=True)
            print(f"prompt-1: {prompt}")
            print(f"default_prompt: {default_prompt}")
            prompt = prompt or default_prompt
            print(f"prompt-2: {prompt}")
            self.chain = prompt | structured_llm

    def process_response(self, document: Document) -> GraphDocument:
        """
        Processes a single document, transforming it into a graph document using
        an LLM based on the model's schema and constraints.
        """
        text = document.page_content
        raw_schema = self.chain.invoke({"input": text})
        if self._function_call:
            raw_schema = cast(Dict[Any, Any], raw_schema)
            nodes, relationships = _convert_to_graph_document(raw_schema)
        else:
            nodes_set = set()
            relationships = []
            parsed_json = self.json_repair.loads(raw_schema.content)
            for rel in parsed_json:
                # Nodes need to be deduplicated using a set
                nodes_set.add((rel["head"], rel["head_type"]))
                nodes_set.add((rel["tail"], rel["tail_type"]))

                source_node = Node(id=rel["head"], type=rel["head_type"])
                target_node = Node(id=rel["tail"], type=rel["tail_type"])
                relationships.append(
                    Relationship(
                        source=source_node, target=target_node, type=rel["relation"]
                    )
                )
            # Create nodes list
            nodes = [Node(id=el[0], type=el[1]) for el in list(nodes_set)]

        # Strict mode filtering
        if self.strict_mode and (self.allowed_nodes or self.allowed_relationships):
            if self.allowed_nodes:
                lower_allowed_nodes = [el.lower() for el in self.allowed_nodes]
                nodes = [
                    node for node in nodes if node.type.lower() in lower_allowed_nodes
                ]
                relationships = [
                    rel
                    for rel in relationships
                    if rel.source.type.lower() in lower_allowed_nodes
                    and rel.target.type.lower() in lower_allowed_nodes
                ]
            if self.allowed_relationships:
                relationships = [
                    rel
                    for rel in relationships
                    if rel.type.lower()
                    in [el.lower() for el in self.allowed_relationships]
                ]

        return GraphDocument(nodes=nodes, relationships=relationships, source=document)

    def convert_to_graph_documents(
        self, documents: Sequence[Document]
    ) -> List[GraphDocument]:
        """Convert a sequence of documents into graph documents.

        Args:
            documents (Sequence[Document]): The original documents.
            **kwargs: Additional keyword arguments.

        Returns:
            Sequence[GraphDocument]: The transformed documents as graphs.
        """
        return [self.process_response(document) for document in documents]

    async def aprocess_response(self, document: Document) -> GraphDocument:
        """
        Asynchronously processes a single document, transforming it into a
        graph document.
        """
        text = document.page_content
        raw_schema = await self.chain.ainvoke({"input": text})
        raw_schema = cast(Dict[Any, Any], raw_schema)
        nodes, relationships = _convert_to_graph_document(raw_schema)

        if self.strict_mode and (self.allowed_nodes or self.allowed_relationships):
            if self.allowed_nodes:
                lower_allowed_nodes = [el.lower() for el in self.allowed_nodes]
                nodes = [
                    node for node in nodes if node.type.lower() in lower_allowed_nodes
                ]
                relationships = [
                    rel
                    for rel in relationships
                    if rel.source.type.lower() in lower_allowed_nodes
                    and rel.target.type.lower() in lower_allowed_nodes
                ]
            if self.allowed_relationships:
                relationships = [
                    rel
                    for rel in relationships
                    if rel.type.lower()
                    in [el.lower() for el in self.allowed_relationships]
                ]

        return GraphDocument(nodes=nodes, relationships=relationships, source=document)

    async def aconvert_to_graph_documents(
        self, documents: Sequence[Document]
    ) -> List[GraphDocument]:
        """
        Asynchronously convert a sequence of documents into graph documents.
        """
        tasks = [
            asyncio.create_task(self.aprocess_response(document))
            for document in documents
        ]
        results = await asyncio.gather(*tasks)
        return results


**Now for the Imports**

This time we are isloating Vertex AI

In [ ]:
import os
import requests

#from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_google_vertexai import ChatVertexAI
from langchain_core.documents import Document

**Diagnostic Methods**

In [ ]:
## Vertex AI Model Graph Transformer Diagnostic Dump
def print_llm_xfrm(llm_xfrm):
    print(f"llm_xfrm: {llm_xfrm}") 
    print(f"allowed_nodes: {llm_xfrm.allowed_nodes}") 
    print(f"allowed_relationships: {llm_xfrm.allowed_relationships}") 
    print(f"strict_mode: {llm_xfrm.strict_mode}")  
    print(f"_function_call: {llm_xfrm._function_call}") 

In [ ]:
## Vertex AI Model Diagnostic Dump
def print_llm(llm):
    print(f"llm: {llm}")
    print(f"name: {llm.model_name}")
    print(f"examples: {llm.examples}") 
    print(f"tuned_model_name: {llm.tuned_model_name}") 
    print(f"convert_system_message_to_human: {llm.convert_system_message_to_human}") 
    print(f"max_output_tokens: {llm.max_output_tokens}") 
    print(f"top_p: {llm.top_p}") 
    print(f"top_k: {llm.top_k}") 
    print(f"credentials: {llm.credentials}")     
    print(f"n: {llm.n}") 
    print(f"streaming: {llm.streaming}") 
    print(f"safety_settings: {llm.safety_settings}")     
    print(f"api_transport: {llm.api_transport}") 
    print(f"api_endpoint: {llm.api_endpoint}") 
 
    print('properties') 
    print('----------') 
    
    print(f"_llm_type: {llm._llm_type}") 
    print(f"is_codey_model: {llm.is_codey_model}")     
    print(f"_is_gemini_model: {llm._is_gemini_model}")
    print(f"_identifying_params: {llm._identifying_params}")     
    print(f"_default_params: {llm._default_params}") 
    print(f"_user_agent: {llm._user_agent}") 



In [70]:
def print_graph_documents(graph_documents):
    print(f"graph_documents_len: {len(graph_documents)}") 
    graphdoc_idx = 0
    for gdoc in graph_documents:
        print(" ") 
        print(f"graphdoc_idx: {graphdoc_idx}") 
        graphdoc_idx += 1
        
        print(f"Len doc_page_content {len(gdoc.source.page_content)}") 
        print(f"No. doc_metadata: {len(gdoc.source.metadata)}")  
        
        print(f"doc_page_content {gdoc.source.page_content}") 
        print(f"doc_metadata: {gdoc.source.metadata}")    
        
        print(f"No. nodes: {len(gdoc.nodes)}") 
        for noddy in gdoc.nodes:
            print(f"Node: {noddy}")
        
        print(f"No. relationships: {len(gdoc.relationships)}") 
        for relly in gdoc.relationships:
            print(f"Relationship: {relly}")
        


**Connect to Google LLMs**

*Least Privilege Security.*

The Notebook is "owned" by a bespoke Service Account created in terrafrom for this purpose.

Minimal permisisons are added (also via terraform) via predefined roles (esp. Vertex) as required.

This is typically triggered by a PERMISSION DENIED error

In [71]:
import os
# Set It - will require regeneration
os.environ['GOOGLE_API_KEY'] = 'd23f54b60a95e062becfc280514f90842e3b8169'
# Access the environment variable later in your code
api_key = os.environ['GOOGLE_API_KEY']
print(api_key)

d23f54b60a95e062becfc280514f90842e3b8169


****Enable Langchain Debugging****

See: https://python.langchain.com/v0.1/docs/guides/development/debugging/

In [83]:
from langchain.globals import set_debug   

set_debug(False)

**Create The LLMs**

Both *Gemini* & *Chat Bison* were created.

Chat Bison malfunctioned so has been abandoned FTTB 

Sourced from here: https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models 

In [73]:
# Avalable Model Variables

# Gemini 1.5 Pro (Preview)
# 404 Publisher Model `projects/nlp-dev-6aae/locations/us-central1/publishers/google/models/gemini-1.5-pro` not found.
#gemini_1pt5_proOnVertex = ChatVertexAI(model="gemini-1.5-pro")
#Retrying langchain_google_vertexai.chat_models._completion_with_retry.<locals>._completion_with_retry_inner in 10.0 seconds as it raised InvalidArgument: 400 Request contains an invalid argument..
gemini_1pt5_proOnVertex = ChatVertexAI(model="gemini-1.5-pro-preview-0409")



# Gemini 1.0 Pro
# This works with Errors - chunking
gemini_1pt0_proOnVertex = ChatVertexAI(model="gemini-1.0-pro")

gemini_proOnVertex = ChatVertexAI(model="gemini-pro")

# PaLM 2 for Chat ("chat-bison")
# This fails atm
model_chat_bison = ChatVertexAI()


**Choose which model we are using**

In [74]:
chat_llm = gemini_1pt0_proOnVertex

*Check Model Properties*

In [75]:
print_llm(chat_llm)

llm: client=<vertexai.generative_models.GenerativeModel object at 0x7fda33d27040> model_name='gemini-1.0-pro' client_preview=<vertexai.generative_models.GenerativeModel object at 0x7fda33d28730>
name: gemini-1.0-pro
examples: None
tuned_model_name: None
convert_system_message_to_human: False
max_output_tokens: None
top_p: None
top_k: None
credentials: None
n: 1
streaming: False
safety_settings: None
api_transport: None
api_endpoint: None
properties
----------
_llm_type: vertexai
is_codey_model: False
_is_gemini_model: True
_identifying_params: {'model_name': 'gemini-1.0-pro', 'candidate_count': 1}
_default_params: {'candidate_count': 1}
_user_agent: langchain-google-vertexai/1.0.3-ChatVertexAI_gemini-1.0-pro


**Construct Graph Transformer**

In [76]:
llm_transformer = LLMGraphTransformer(llm=chat_llm)
#llm_transformer._function_call = False # Causes Error
## Shows how to override prompt
#llm_transformer = LLMGraphTransformer(llm=chat_llm, prompt=default_prompt)

prompt-1: None
default_prompt: input_variables=['input'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='# Knowledge Graph Instructions for Gemini\n## 1. Overview\nYou are a top-tier algorithm designed for extracting information in structured formats to build a knowledge graph.\nTry to capture as much information from the text as possible without sacrifing accuracy. Do not add any information that is not explicitly mentioned in the text\n- **Nodes** represent entities and concepts.\n- The aim is to achieve simplicity and clarity in the knowledge graph, making it\naccessible for a vast audience.\n## 2. Labeling Nodes\n- **Consistency**: Ensure you use available types for node labels.\nEnsure you use basic or elementary types for node labels.\n- For example, when you identify an entity representing a person, always label it as **\'person\'**. Avoid using more specific terms like \'mathematician\' or \'scientist\'  - **Node IDs**: Never utilize in

In [77]:
print_llm_xfrm(llm_transformer)

llm_xfrm: <__main__.LLMGraphTransformer object at 0x7fda33d2b3a0>
allowed_nodes: []
allowed_relationships: []
strict_mode: True
_function_call: True


**Easy Test Case**

In [78]:
from langchain_core.documents import Document

text = """
Current Professional Machine Learning Engineer Certification exam guide
A Professional Machine Learning Engineer builds, evaluates, productionizes, and optimizes ML models by using Google Cloud technologies and knowledge of proven models and techniques. The ML Engineer handles large, complex datasets and creates repeatable, reusable code. The ML Engineer considers responsible AI and fairness throughout the ML model development process, and collaborates closely with other job roles to ensure long-term success of ML-based applications. The ML Engineer has strong programming skills and experience with data platforms and distributed data processing tools. The ML Engineer is proficient in the areas of model architecture, data and ML pipeline creation, and metrics interpretation. The ML Engineer is familiar with foundational concepts of MLOps, application development, infrastructure management, data engineering, and data governance. The ML Engineer makes ML accessible and enables teams across the organization. By training, retraining, deploying, scheduling, monitoring, and improving models, the ML Engineer designs and creates scalable, performant solutions.
Note: The exam does not directly assess coding skill. If you have a minimum proficiency in Python and Cloud SQL, you should be able to interpret any questions with code snippets.
Register now
The Professional Machine Learning Engineer exam does not cover generative AI, as the tools used to develop generative AI-based solutions are evolving quickly. If you are interested in generative AI, please refer to the Introduction to Generative AI Learning Path (all audiences) or the Generative AI for Developers Learning Path (technical audience). If you are a partner, please refer to the Gen AI partner courses: Introduction to Generative AI Learning Path, Generative AI for ML Engineers, and Generative AI for Developers.
Section 1: Architecting low-code ML solutions (~12% of the exam)
1.1 Developing ML models by using BigQuery ML. Considerations include:
Building the appropriate BigQuery ML model (e.g., linear and binary classification, regression, time-series, matrix factorization, boosted trees, autoencoders) based on the business problem
Feature engineering or selection by using BigQuery ML
Generating predictions by using BigQuery ML
1.2 Building AI solutions by using ML APIs. Considerations include:
Building applications by using ML APIs (e.g., Cloud Vision API, Natural Language API, Cloud Speech API, Translation)
Building applications by using industry-specific APIs (e.g., Document AI API, Retail API)
1.3 Training models by using AutoML. Considerations include:
Preparing data for AutoML (e.g., feature selection, data labeling, Tabular Workflows on AutoML)
Using available data (e.g., tabular, text, speech, images, videos) to train custom models
Using AutoML for tabular data
Creating forecasting models using AutoML
Configuring and debugging trained models
Section 2: Collaborating within and across teams to manage data and models (~16% of the exam)
2.1 Exploring and preprocessing organization-wide data (e.g., Cloud Storage, BigQuery, Spanner, Cloud SQL, Apache Spark, Apache Hadoop). Considerations include:
Organizing different types of data (e.g., tabular, text, speech, images, videos) for efficient training
Managing datasets in Vertex AI
Data preprocessing (e.g., Dataflow, TensorFlow Extended [TFX], BigQuery)
Creating and consolidating features in Vertex AI Feature Store
Privacy implications of data usage and/or collection (e.g., handling sensitive data such as personally identifiable information [PII] and protected health information [PHI])
2.2 Model prototyping using Jupyter notebooks. Considerations include:
Choosing the appropriate Jupyter backend on Google Cloud (e.g., Vertex AI Workbench, notebooks on Dataproc)
Applying security best practices in Vertex AI Workbench
Using Spark kernels
Integration with code source repositories
Developing models in Vertex AI Workbench by using common frameworks (e.g., TensorFlow, PyTorch, sklearn, Spark, JAX)
2.3 Tracking and running ML experiments. Considerations include:
Choosing the appropriate Google Cloud environment for development and experimentation (e.g., Vertex AI Experiments, Kubeflow Pipelines, Vertex AI TensorBoard with TensorFlow and PyTorch) given the framework
Section 3: Scaling prototypes into ML models (~18% of the exam)
3.1 Building models. Considerations include:
Choosing ML framework and model architecture
Modeling techniques given interpretability requirements
3.2 Training models. Considerations include:
Organizing training data (e.g., tabular, text, speech, images, videos) on Google Cloud (e.g., Cloud Storage, BigQuery)
Ingestion of various file types (e.g., CSV, JSON, images, Hadoop, databases) into training
Training using different SDKs (e.g., Vertex AI custom training, Kubeflow on Google Kubernetes Engine, AutoML, tabular workflows)
Using distributed training to organize reliable pipelines
Hyperparameter tuning
Troubleshooting ML model training failures
3.3 Choosing appropriate hardware for training. Considerations include:
Evaluation of compute and accelerator options (e.g., CPU, GPU, TPU, edge devices)
Distributed training with TPUs and GPUs (e.g., Reduction Server on Vertex AI, Horovod)
Section 4: Serving and scaling models (~19% of the exam)
4.1 Serving models. Considerations include:
Batch and online inference (e.g., Vertex AI, Dataflow, BigQuery ML, Dataproc)
Using different frameworks (e.g., PyTorch, XGBoost) to serve models
Organizing a model registry
A/B testing different versions of a model
4.2 Scaling online model serving. Considerations include:
Vertex AI Feature Store
Vertex AI public and private endpoints
Choosing appropriate hardware (e.g., CPU, GPU, TPU, edge)
Scaling the serving backend based on the throughput (e.g., Vertex AI Prediction, containerized serving)
Tuning ML models for training and serving in production (e.g., simplification techniques, optimizing the ML solution for increased performance, latency, memory, throughput)
Section 5: Automating and orchestrating ML pipelines (~21% of the exam)
5.1 Developing end-to-end ML pipelines. Considerations include:
Data and model validation
Ensuring consistent data pre-processing between training and serving
Hosting third-party pipelines on Google Cloud (e.g., MLFlow)
Identifying components, parameters, triggers, and compute needs (e.g., Cloud Build, Cloud Run)
Orchestration framework (e.g., Kubeflow Pipelines, Vertex AI Pipelines, Cloud Composer)
Hybrid or multicloud strategies
System design with TFX components or Kubeflow DSL (e.g., Dataflow)
5.2 Automating model retraining. Considerations include:
Determining an appropriate retraining policy
Continuous integration and continuous delivery (CI/CD) model deployment (e.g., Cloud Build, Jenkins)
5.3 Tracking and auditing metadata. Considerations include: 
Tracking and comparing model artifacts and versions (e.g., Vertex AI Experiments, Vertex ML Metadata)
Hooking into model and dataset versioning
Model and data lineage
Section 6: Monitoring ML solutions (~14% of the exam)
6.1 Identifying risks to ML solutions. Considerations include:
Building secure ML systems (e.g., protecting against unintentional exploitation of data or models, hacking)
Aligning with Google’s Responsible AI practices (e.g., biases)
Assessing ML solution readiness (e.g., data bias, fairness)
Model explainability on Vertex AI (e.g., Vertex AI Prediction)
6.2 Monitoring, testing, and troubleshooting ML solutions. Considerations include:
Establishing continuous evaluation metrics (e.g., Vertex AI Model Monitoring, Explainable AI)
Monitoring for training-serving skew
Monitoring for feature attribution drift
Monitoring model performance against baselines, simpler models, and across the time dimension
Common training and serving errors
"""
documents = [Document(page_content=text)]

In [79]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# split documents into text and embeddings

text_splitter = RecursiveCharacterTextSplitter(
   chunk_size=512, 
   chunk_overlap=20,
   length_function=len,
   is_separator_regex=False
)

chunks = text_splitter.split_documents(documents)

print(f"No. chunks{len(chunks)}")

No. chunks20


**Use Gemini** 

Works but with intermittent error which may cause data loss.

Hence need for chunking per variouis discisions forums &c.  

In [84]:
graph_documents = llm_transformer.convert_to_graph_documents(chunks)

Check content of **Graph Documents**

In [85]:
print_graph_documents(graph_documents)

graph_documents_len: 20
 
graphdoc_idx: 0
Len doc_page_content 71
No. doc_metadata: 0
doc_page_content Current Professional Machine Learning Engineer Certification exam guide
doc_metadata: {}
No. nodes: 1
Node: id='Current Professional Machine Learning Engineer Certification Exam Guide' type='Document'
No. relationships: 0
 
graphdoc_idx: 1
Len doc_page_content 506
No. doc_metadata: 0
doc_page_content A Professional Machine Learning Engineer builds, evaluates, productionizes, and optimizes ML models by using Google Cloud technologies and knowledge of proven models and techniques. The ML Engineer handles large, complex datasets and creates repeatable, reusable code. The ML Engineer considers responsible AI and fairness throughout the ML model development process, and collaborates closely with other job roles to ensure long-term success of ML-based applications. The ML Engineer has strong programming
doc_metadata: {}
No. nodes: 9
Node: id='Machine Learning Engineer' type='Job title'
Node

**Enter Node4J**

Node4J Connectivity

Requires singing up for free version.

DB Will be stopped if not recently used and will require resuming else will fail. 

In [86]:
import os

from langchain_community.graphs import Neo4jGraph

os.environ["NEO4J_URI"] = "neo4j+s://a657168d.databases.neo4j.io"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "VM3A9Mz6usNT99nLs_lqQssfVK8JxeD81DnEiXlDkZU"

graph = Neo4jGraph()

**Add to GraphDB**

This statement loads Nodes & Relatonships into Node4J

Thence they can be viewed/manipulated directly on the DB. 

In [87]:
graph.add_graph_documents(graph_documents)